## YaTC embeddings calculation

In [ ]:
%cd ../models/YaTC/src

/pscratch/sd/k/kell/demystifying/YaTC/src


In [2]:
import pickle

In [3]:
from tqdm.auto import tqdm
import copy
import threading

from finetune import build_dataset
import torch
import models_YaTC
from util.pos_embed import interpolate_pos_embed
from tqdm import tqdm
from collections import defaultdict

In [ ]:
def get_embeddings(datafolder, n_classes, batch_size=64, limit = 10**30, gpus=4):
    loader_yatc = lambda: True
    loader_yatc.data_path = datafolder
    loader_yatc = build_dataset(is_train=True, args=loader_yatc)
    loader_yatc = torch.utils.data.DataLoader(
            loader_yatc, sampler=torch.utils.data.SequentialSampler(loader_yatc),
            batch_size=batch_size,
            num_workers=4,
            pin_memory=False,
            drop_last=True,
        )

    checkpoint_model = torch.load("../models/YaTC/YaTC_pretrained_model.pth")['model']
    # yes without the 's'
    yatc_frozen_model = models_YaTC.__dict__['TraFormer_YaTC'](
            num_classes=n_classes,
            drop_path_rate=0.1,
        )
    interpolate_pos_embed(yatc_frozen_model, checkpoint_model)

    #rename norm to fc_norm and delete extra keys
    checkpoint_model['fc_norm.bias'] = checkpoint_model['norm.bias']
    checkpoint_model['fc_norm.weight'] = checkpoint_model['norm.weight']

    keys_to_del = ['mask_token', 'norm.weight', 'norm.bias']
    for key in checkpoint_model.keys():
        if key.startswith('decoder'):
            keys_to_del.append(key)

    for key in keys_to_del:
        del checkpoint_model[key]

    yatc_frozen_model.load_state_dict(checkpoint_model, strict=False)

    yatc_models = {}
    for i in range(gpus):
        yatc_models[i] = copy.deepcopy(yatc_frozen_model)
        yatc_models[i].to(f"cuda:{i}")

    print(f"Total: {len(loader_yatc)}")

    def encode_and_append(batch, model, result_list, i):
        imgs, _ = batch
        imgs = imgs.to(f"cuda:{i}")
        with torch.no_grad():
            result_list.append(model.forward_features(imgs).mean(dim=1).cpu())
        del imgs
        del batch

    counter = 0
    result_embeddings = []
    result_filenames = []
    with torch.no_grad():
        iterator = iter(loader_yatc)
        try:
            for y in tqdm(range(0, min(len(loader_yatc) // gpus, limit))):
                yatc_emb = defaultdict(list)
                batches = [next(iterator) for i in range(gpus)]
                threads = []
                for i in range(gpus):
                    t = threading.Thread(target=encode_and_append, args=(batches[i], yatc_models[i], yatc_emb[i], i))
                    t.start()
                    threads.append(t)
                for t in threads:
                    t.join()
                del batches
                result_embeddings.append(torch.cat([torch.cat(yatc_emb[i]) for i in yatc_emb]))
                result_filenames.extend([x[0] for x in loader_yatc.dataset.samples[y*gpus*batch_size:(y+1)*gpus*batch_size]])
                
        except StopIteration:
            print("finished")
        except Exception as e:
            print(e)

    return torch.cat(result_embeddings), result_filenames

In [ ]:
caida_emb = get_embeddings("../data/newYaTC/", n_classes=4, batch_size=1, gpus=1)
with open("../data/newYaTC/synth_emb.pkl", "bw") as f:
    pickle.dump(caida_emb, f)

Dataset ImageFolder
    Number of datapoints: 367
    Root location: /pscratch/sd/k/kell/demystifying/data/newYaTC/train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               ToTensor()
               Normalize(mean=[0.5], std=[0.5])
           )
Total: 367


100%|██████████| 367/367 [00:06<00:00, 60.23it/s]


In [ ]:
caida_emb = get_embeddings("../data/caida/", n_classes=1, batch_size=512)
with open("../data/caida_emb.pkl", "bw") as f:
    pickle.dump(caida_emb, f)

FileNotFoundError: [Errno 2] No such file or directory: '/dev/shm/data/caida/train'

In [ ]:
caida_emb = get_embeddings("../data/cicapt/", n_classes=22, batch_size=512)
with open("../data/cicapt_emb.pkl", "bw") as f:
    pickle.dump(caida_emb, f)

Dataset ImageFolder
    Number of datapoints: 1578213
    Root location: /dev/shm/data/cicapt/train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               ToTensor()
               Normalize(mean=[0.5], std=[0.5])
           )
Total: 3082


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa5f6fdf700>
Traceback (most recent call last):
  File "/global/common/software/m4629/environments/condaenv/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "/global/common/software/m4629/environments/condaenv/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
    if w.is_alive():
  File "/global/common/software/m4629/environments/condaenv/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa5f6fdf700>
Traceback (most recent call last):
  File "/global/common/software/m4629/environments/condaenv/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shut

In [ ]:
cicids_emb = get_embeddings("../data/cicids/", n_classes=8, batch_size=512)
with open("../data/cicids_emb.pkl", "bw") as f:
    pickle.dump(cicids_emb, f)

Dataset ImageFolder
    Number of datapoints: 558884
    Root location: /dev/shm/data/cicids/train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               ToTensor()
               Normalize(mean=[0.5], std=[0.5])
           )
Total: 1091


100%|██████████| 272/272 [00:41<00:00,  6.49it/s]


In [ ]:
cross_emb = get_embeddings("../data/cross/", n_classes=210, batch_size=512)
with open("../data/cross_emb.pkl", "bw") as f:
    pickle.dump(cross_emb, f)

Dataset ImageFolder
    Number of datapoints: 44647
    Root location: /dev/shm/data/cross/train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               ToTensor()
               Normalize(mean=[0.5], std=[0.5])
           )
Total: 87


100%|██████████| 21/21 [00:03<00:00,  6.23it/s]


In [ ]:
mawi_emb = get_embeddings("../data/mawi/", n_classes=1, batch_size=512)
with open("../data/mawi_emb.pkl", "bw") as f:
    pickle.dump(mawi_emb, f)

Dataset ImageFolder
    Number of datapoints: 999954
    Root location: /dev/shm/data/mawi/train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               ToTensor()
               Normalize(mean=[0.5], std=[0.5])
           )
Total: 1953


100%|██████████| 488/488 [01:12<00:00,  6.70it/s]


In [ ]:
emb = get_embeddings("../data/yatc/", n_classes=1, batch_size=1, gpus=1)
with open("../data/synthstability_emb.pkl", "bw") as f:
    pickle.dump(emb, f)

Dataset ImageFolder
    Number of datapoints: 10
    Root location: /dev/shm/data/yatc/train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               ToTensor()
               Normalize(mean=[0.5], std=[0.5])
           )
Total: 10


100%|██████████| 10/10 [00:00<00:00, 55.54it/s]


In [ ]:
emb = get_embeddings("../data/synth/", n_classes=5, batch_size=1, gpus=1)
with open("../data/synth_emb.pkl", "bw") as f:
    pickle.dump(emb, f)

Dataset ImageFolder
    Number of datapoints: 472
    Root location: /dev/shm/data/synth/train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               ToTensor()
               Normalize(mean=[0.5], std=[0.5])
           )
Total: 472


100%|██████████| 472/472 [00:08<00:00, 55.28it/s]


In [ ]:
emb = get_embeddings("../data/perf/", n_classes=4, batch_size=1, gpus=1)
with open("../data/perf_emb.pkl", "bw") as f:
    pickle.dump(emb, f)

Dataset ImageFolder
    Number of datapoints: 40
    Root location: /dev/shm/data/perf/train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               ToTensor()
               Normalize(mean=[0.5], std=[0.5])
           )
Total: 40


100%|██████████| 40/40 [00:01<00:00, 20.99it/s]
